In [1]:
from MyMessage import *
import time, socket

# ClientSide

In [2]:
def findHost(serverName:str, numberIntervals : int = 5, intervalWaitTime : float = 1, isLocal : bool = True)->str:
    serverIP : str= None
    for _ in range(numberIntervals):
        if _ == numberIntervals -1:
            print("Last try before giving up")
        try:
            serverIP = None
            if isLocal:
                serverIP = socket.gethostbyname(serverName + '.local')
            else:
                serverIP = socket.gethostbyname(serverName)
            print(f"{serverName} has been found at {serverIP}")
            return serverIP
            
        except socket.gaierror:
            print(f"{serverName} has not been found. Waiting {intervalWaitTime} seconds before retrying.")
            time.sleep(intervalWaitTime)
    return serverIP

In [3]:
def getConnected(serverIP : str, clientsocket,  port : int = 5050, numberIntervals : int = 5,  intervalWaitTime : float = 1):
    for _ in range(numberIntervals):
        if _ == numberIntervals -1:
            print("Last try before giving up")
        else:
            print("Server has refused to connect. Trying again")
        try:
            clientsocket.connect((serverIP,5050))
            print("Connected to server")
            return clientsocket
        except ConnectionRefusedError:
            time.sleep(intervalWaitTime)
            
    print("System has failed to connect with the server.")
    return None

In [4]:
class ClientSocket:
    def __init__(self, server_NameOrIP : str, portToConnectTo : int = 5050, isLocal : bool = True):
        self.__socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.__port = 5050
        self.__clientMessager = MyMessage({"endc":self.__closeSocket})
        self.__serverName : str = ""
        self.__serverIP : str = ""
        self.__findServer(server_NameOrIP, isLocal)
        if getConnected(self.__serverIP, self.__socket, self.__port , 5, 0.5) is None:
            raise ConnectionRefusedError()

    def __del__(self):
        self.endCommunication()
        
    def __findServer(self, server_NameOrIP : str, isLocal : bool):
        if server_NameOrIP[0].isdigit():
            self.__serverName = socket.gethostbyaddr(clientAddress)
            self.__serverIP = server_NameOrIP
        else:
            self.__serverName = server_NameOrIP
            self.__serverIP = findHost(server_NameOrIP, 5, 0.5, isLocal)
            if self.__serverIP == None:
                raise ConnectionRefusedError()

    def __closeSocket(self, data):
        pass

#----------------------------------------------------------
    def endCommunication(self):
        self.__clientMessager.command("endc", None)
        for message, encoding in self.__clientMessager.nextMessage():
            self.__socket.send(message.encode(encoding))

    def echo(self, message : str):
        self.__clientMessager.echo(message)
        for message, encoding in self.__clientMessager.nextMessage():
            self.__socket.send(sendMessage.encode(encoding))

    def queCommand(self, command : str, data):
        self.__clientMessager.command(command, data) 
        
    def sendNext(self):
        try:
            message, encoding = next(self.__serverMessager.nextMessage())
            self.__serverSocket.send(message.encode(encoding))
            return True
        except StopIteration:
            return False

    def recv(self):
        messageSize, encoding = next(self.__clientMessager.nextMessage())
        recievedMessage = self.__socket.recv(messageSize).decode(encoding)
        print(recievedMessage)
        return self.__clientMessager.recv(recievedMessage, messageEncoding)
        
        
        
        

In [5]:
def startClient(serverName : str, port =5050):
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    clientMessager = MyMessage()
    serverIP = findLocalHost(serverName,intervalWaitTime=0.5)
    if serverIP == None:
        return None
    toServer = getConnected(serverIP, client, port)
    if toServer == None:
        return None
    clientMessager.echo("Hello World")
    clientMessager.command("endT", None)
    sendMessage, encoding = next(clientMessager.nextMessage())
    print(f"message being sent is {sendMessage}")
    toServer.send(sendMessage.encode(encoding))
    

# ServerSide

In [6]:
class ListeningServer:
    def __init__(self, port : int, amountOfClients : int = 1): #amount of clients can be connected to the listening server at once
        
        self.__listeningPort : int = port
        self.__hostname : int = socket.gethostname()
        self.__hostIP : int = socket.gethostbyname(self.__hostname)
        
        self.__listeningSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.__listeningSocket.bind((self.__hostIP, self.__listeningPort))
        self.__listeningSocket.listen(amountOfClients)

    def __del__(self):
        self.__listeningSocket.close()
    
    def getSocket(self) -> dict:
        communicationSocket, clientAddress = self.__listeningSocket.accept() #becuase client address and port is a tuple? ¯\_(ツ)_/¯
        clientAddress, clientPort = clientAddress
        
        clientName = socket.gethostbyaddr(clientAddress)
        return communicationSocket, {"client name":clientName, "client address": clientAddress, "client port":clientPort}

In [20]:
class HostingServer:
    def __init__(self, hostingSocket, clientInfo : dict = {}):
        self.__serverSocket = hostingSocket
        self.__clientInfo : dict = clientInfo
        self.__serverMessager = MyMessage({"endc":self.__closeSocket})
    
    def __del__(self):
        self.endCommunication()
        self.__serverSocket.close()
        self.__clientInfo = None

    def __closeSocket(self, data):
        self.__serverSocket.close()
        self.__serverSocket = None
        self.__clientInfo = None
        return True, None
#--------------------------------------------------------
    def endCommunication(self):
        self.__serverMessager.command("endc", None)
        for message, encoding in self.__serverMessager.nextMessage():
            self.__serverSocket.send(message.encode(encoding))
        
        self.__closeSocket(None)

    def echo(self, message : str):
        self.__serverMessager.echo(message)

    def queCommand(self, command : str, data):
        self.__serverMessager.command(command, data)
        
    def sendNext(self):
        try:
            message, encoding = next(self.__serverMessager.nextMessage())
            self.__serverSocket.send(message.encode(encoding))
            return True
        except StopIteration:
            return False
        
    def recv(self):
        if self.__serverSocket is None:
            raise TypeError()
            return None
        messageSize, messageEncoding = self.__serverMessager.nextMessageFormat()
        recievedMessage = self.__serverSocket.recv(messageSize).decode(messageEncoding)
        print(recievedMessage)
        return self.__serverMessager.recv(recievedMessage, messageEncoding)
    

In [8]:
def getServerInfo():
    return {"hostname" : socket.gethostname(), "hostIP" : socket.gethostbyname(socket.gethostname())}

In [9]:
def startServer(port : int = 5050):
    serverInfo = getServerInfo
    serverBeginingTime = time.time()
    
    
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((hostIP,port))
    server.listen(5)
    while True:
        communication_socket, clientAddress = server.accept()
        clientname = socket.gethostbyaddr(clientAddress)[0]
        print(f"{clientname} @ {clientAddress} connected")
        server.close()
    

In [10]:
def closeConnection(data = None):
    pass

In [11]:
eventFunctions = {"endC":closeConnection} #sends close connection message

# EventFunctions

# Testing

In [21]:
test = ClientSocket(socket.gethostname())

ConsensualEcstasy has been found at 192.168.56.1
Server has refused to connect. Trying again
Connected to server


In [22]:
test.endCommunication()